In [340]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [341]:
job_desc_ca = pd.read_csv('../../../data/raw_data/elisa_position_ca.csv')
job_desc_ca = job_desc_ca.drop(['Unnamed: 0', 'Sector', 'Position'], axis=1)
job_desc_ca = job_desc_ca.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))
job_desc_ca = job_desc_ca.dropna(subset=['Position_clean'])

In [342]:
job_desc_us = pd.read_csv('../../../data/raw_data/elisa_position_us.csv')
job_desc_us = job_desc_us.drop(['Unnamed: 0', 'Position', 'Sector'], axis=1)
job_desc_us = job_desc_us.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))
job_desc_us = job_desc_us.dropna(subset=['Position_clean'])

In [363]:
job_desc_uk = pd.read_csv('../../../data/raw_data/elisa_position_uk.csv')
job_desc_uk = job_desc_uk.drop(['Unnamed: 0', 'Sector', 'Position'], axis=1)
job_desc_uk = job_desc_uk.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))
job_desc_uk = job_desc_uk.dropna(subset=['Position_clean'])

In [344]:
job_desc_ca = job_desc_ca.rename({'Company': 'company', 'Industry': 'company_industry', \
                                  'Location': 'company_location', 'Rating': 'company_rating', \
                                  'Revenue': 'company_revenue', 'Size': 'company_size', \
                                  'Type': 'company_type', 'Year Founded': 'company_year_founded'}, axis=1)
job_desc_us = job_desc_us.rename({'Company': 'company', 'Industry': 'company_industry', \
                                  'Location': 'company_location', 'Rating': 'company_rating', \
                                  'Revenue': 'company_revenue', 'Size': 'company_size', \
                                  'Type': 'company_type', 'Year Founded': 'company_year_founded'}, axis=1)
job_desc_uk = job_desc_uk.rename({'Company': 'company', 'Industry': 'company_industry', \
                                  'Location': 'company_location', 'Rating': 'company_rating', \
                                  'Revenue': 'company_revenue', 'Size': 'company_size', \
                                  'Type': 'company_type', 'Year Founded': 'company_year_founded'}, axis=1)

In [345]:
company_info_ca = pd.read_csv('../../../data/raw_data/companies_ca_info.csv')
company_info_ca = company_info_ca.drop(['Unnamed: 0', 'index'], axis=1)
company_info_ca = company_info_ca.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [346]:
company_info_us = pd.read_csv('../../../data/raw_data/companies_us_info.csv')
company_info_us = company_info_us.drop(['Unnamed: 0', 'index'], axis=1)
company_info_us = company_info_us.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [347]:
company_info_uk = pd.read_csv('../../../data/raw_data/companies_uk_info.csv')
company_info_uk = company_info_uk.drop(['Unnamed: 0'], axis=1)
company_info_uk = company_info_uk.replace(to_replace=['NF', 'Unknown', '-1', -1, 'Unknown / Non-Applicable'], value=float('nan'))

In [348]:
def get_unique_vals(column_name):
    unique_1 = job_desc_us[column_name].unique()
    unique_1 = [x for x in unique_1 if str(x) != 'nan']
    unique_2 = job_desc_ca[column_name].unique()
    unique_2 = [x for x in unique_2 if str(x) != 'nan']
    unique_3 = job_desc_uk[column_name].unique()
    unique_3 = [x for x in unique_3 if str(x) != 'nan']
    
    unique_4 = company_info_us[column_name].unique()
    unique_4 = [x for x in unique_4 if str(x) != 'nan']
    unique_5 = company_info_ca[column_name].unique()
    unique_5 = [x for x in unique_5 if str(x) != 'nan']
    unique_6 = company_info_uk[column_name].unique()
    unique_6 = [x for x in unique_6 if str(x) != 'nan']
    
    results = [unique_1, unique_2, unique_3, unique_4, unique_5, unique_6]
    results_union = set().union(*results)
    return (unique_1, unique_2, unique_3, unique_4, unique_5, unique_6, results_union)

In [349]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

In [350]:
def replace_col_with_categorical(df, col_name, cat):
    df = df.drop(col_name, axis=1)
    df[cat.columns] = cat
    return df

In [351]:
def convert_to_categorical(df, column_name, unique_col_vals, unique_vals):
    company = pd.get_dummies(df[column_name], prefix=column_name)
    dif = Diff(unique_col_vals, unique_vals)
    df = replace_col_with_categorical(df, column_name, company)
    for col in dif:
        df[col] = 0
    return df

In [352]:
def convert_all_dfs_to_categorical(column_name):
    unique_1, unique_2, unique_3, unique_4, unique_5, unique_6, results_union = get_unique_vals(column_name)
    job_us = convert_to_categorical(job_desc_us, column_name, unique_1, results_union)
    job_ca = convert_to_categorical(job_desc_ca, column_name, unique_2, results_union)
    job_uk = convert_to_categorical(job_desc_uk, column_name, unique_3, results_union)

    company_us = convert_to_categorical(company_info_us, column_name, unique_4, results_union)
    company_ca = convert_to_categorical(company_info_ca, column_name, unique_5, results_union)
    company_uk = convert_to_categorical(company_info_uk, column_name, unique_6, results_union)
    return (job_us, job_ca, job_uk, company_us, company_ca, company_uk)

### Type, Size, Industry -> convert to categorical

In [353]:
job_desc_us, job_desc_ca, job_desc_uk, company_info_us, company_info_ca, company_info_uk = convert_all_dfs_to_categorical('company_industry')
job_desc_us, job_desc_ca, job_desc_uk, company_info_us, company_info_ca, company_info_uk = convert_all_dfs_to_categorical('company_revenue')
job_desc_us, job_desc_ca, job_desc_uk, company_info_us, company_info_ca, company_info_uk = convert_all_dfs_to_categorical('company_size')
job_desc_us, job_desc_ca, job_desc_uk, company_info_us, company_info_ca, company_info_uk = convert_all_dfs_to_categorical('company_type')

### Clean Salary Estimate Column

In [356]:
def is_range(salary):
    if type(salary) != float:
        is_range = salary.find('-')
        return is_range

In [357]:
def clean_salary(salary):
    if type(salary) != float:
        ran = is_range(salary)
        salary = salary.replace("CA", "")
        salary = salary.replace("€", "")
        salary = salary.replace("₹", "")
        salary = salary.replace("$", "")
        salary = salary.replace("£", "")
        salary = salary.replace("₪", "")
        salary = salary.replace("K", "000")
        salary = salary.replace("k", "000")
        salary = salary.replace(",", "")
        salary = salary.replace("(Glassdoor Est.)", "")
        salary = salary.replace("(Employer Est.)", "")
        is_hourly = salary.find("Per Hour")
        salary = salary.replace("Per Hour", "")
        if is_hourly != -1:
            lower = int(salary[:ran].replace("-", ""))
            upper = int(salary[ran:])
            salary = ((lower + upper) / 2) * 40 * 52
        elif (ran != -1) and (is_hourly == -1):
            lower = int(salary[:ran + 1].replace("-", ""))
            upper = int(salary[ran + 2:])
            salary = (lower + upper) / 2
        else:
            try:
                salary = int(salary)
            except:
                salary = float('nan')
    return salary

In [358]:
def convert_usd_to_cad(usd):
    try:
        return usd * 1.31
    except:
        return float('nan')

In [359]:
def convert_pound_to_cad(pound):
    try:
        return pound * 1.74
    except:
        return float('nan')

In [360]:
job_desc_us['Glassdoor Salary Estimate'] = job_desc_us['Glassdoor Salary Estimate'].map(clean_salary).map(convert_usd_to_cad)
job_desc_ca['Glassdoor Salary Estimate'] = job_desc_ca['Glassdoor Salary Estimate'].map(clean_salary)
job_desc_uk['Glassdoor Salary Estimate'] = job_desc_ca['Glassdoor Salary Estimate'].map(clean_salary).map(convert_pound_to_cad)

In [365]:
# Write to files
job_desc_us.to_csv('../../../data/clean_data/job_desc_us.csv')
job_desc_ca.to_csv('../../../data/clean_data/job_desc_ca.csv')
job_desc_uk.to_csv('../../../data/clean_data/job_desc_uk.csv')

company_info_us.to_csv('../../../data/clean_data/company_info_us.csv')
company_info_ca.to_csv('../../../data/clean_data/company_info_ca.csv')
company_info_uk.to_csv('../../../data/clean_data/company_info_uk.csv')